In [13]:
import random
import hashlib
from typing import Tuple

In [14]:
PRIME = 23 
GENERATOR = 5  

print(f"\n{'='*80}")
print("PUBLIC PARAMETERS (Known to everyone)")
print(f"{'='*80}")
print(f"Prime (p): {PRIME}")
print(f"Generator (g): {GENERATOR}")


PUBLIC PARAMETERS (Known to everyone)
Prime (p): 23
Generator (g): 5


In [15]:
def generate_private_key(max_value: int = 100) -> int:
    """Generate a random private key"""
    return random.randint(2, max_value)

def generate_public_key(private_key: int, generator: int, prime: int) -> int:
    """Generate public key using: (generator^private_key) mod prime"""
    return pow(generator, private_key, prime)

def generate_shared_secret(other_public_key: int, own_private_key: int, prime: int) -> int:
    """Generate shared secret using: (other_public_key^own_private_key) mod prime"""
    return pow(other_public_key, own_private_key, prime)

In [16]:
print(f"\n{'='*80}")
print("ALICE'S KEY GENERATION")
print(f"{'='*80}")

# Alice generates her private key
alice_private_key = generate_private_key()
print(f"Alice's private key (a): {alice_private_key} [SECRET - kept private]")

# Alice computes her public key: A = g^a mod p
alice_public_key = generate_public_key(alice_private_key, GENERATOR, PRIME)
print(f"Alice's public key (A): {alice_public_key} [PUBLIC - will be sent to Bob]")
print(f"Calculation: A = {GENERATOR}^{alice_private_key} mod {PRIME} = {alice_public_key}")


ALICE'S KEY GENERATION
Alice's private key (a): 51 [SECRET - kept private]
Alice's public key (A): 17 [PUBLIC - will be sent to Bob]
Calculation: A = 5^51 mod 23 = 17


In [17]:
print(f"\n{'='*80}")
print("BOB'S KEY GENERATION")
print(f"{'='*80}")

# Bob generates his private key
bob_private_key = generate_private_key()
print(f"Bob's private key (b): {bob_private_key} [SECRET - kept private]")

# Bob computes his public key: B = g^b mod p
bob_public_key = generate_public_key(bob_private_key, GENERATOR, PRIME)
print(f"Bob's public key (B): {bob_public_key} [PUBLIC - will be sent to Alice]")
print(f"Calculation: B = {GENERATOR}^{bob_private_key} mod {PRIME} = {bob_public_key}")


BOB'S KEY GENERATION
Bob's private key (b): 48 [SECRET - kept private]
Bob's public key (B): 4 [PUBLIC - will be sent to Alice]
Calculation: B = 5^48 mod 23 = 4


In [18]:
print(f"\n{'='*80}")
print("TOM (ATTACKER) KEY GENERATION")
print(f"{'='*80}")

# Tom (the attacker) generates his own private key
tom_private_key = generate_private_key()
print(f"Tom's private key (t): {tom_private_key} [SECRET - Tom's key]")

# Tom computes his public key: T = g^t mod p
tom_public_key = generate_public_key(tom_private_key, GENERATOR, PRIME)
print(f"Tom's public key (T): {tom_public_key} [PUBLIC - Tom will use this for MITM]")
print(f"Calculation: T = {GENERATOR}^{tom_private_key} mod {PRIME} = {tom_public_key}")


TOM (ATTACKER) KEY GENERATION
Tom's private key (t): 54 [SECRET - Tom's key]
Tom's public key (T): 9 [PUBLIC - Tom will use this for MITM]
Calculation: T = 5^54 mod 23 = 9


In [19]:
print(f"\n{'='*80}")
print("MAN-IN-THE-MIDDLE ATTACK - KEY EXCHANGE INTERCEPTION")
print(f"{'='*80}")

print("\n--- STEP 1: Alice tries to send her public key to Bob ---")
print(f"Alice sends: A = {alice_public_key}")
print(f"❌ Tom INTERCEPTS this message!")
print(f"✓ Tom REPLACES it with his own public key: T = {tom_public_key}")
print(f"→ Bob receives: {tom_public_key} (thinks it's from Alice)")

print("\n--- STEP 2: Bob tries to send his public key to Alice ---")
print(f"Bob sends: B = {bob_public_key}")
print(f"❌ Tom INTERCEPTS this message!")
print(f"✓ Tom REPLACES it with his own public key: T = {tom_public_key}")
print(f"→ Alice receives: {tom_public_key} (thinks it's from Bob)")


MAN-IN-THE-MIDDLE ATTACK - KEY EXCHANGE INTERCEPTION

--- STEP 1: Alice tries to send her public key to Bob ---
Alice sends: A = 17
❌ Tom INTERCEPTS this message!
✓ Tom REPLACES it with his own public key: T = 9
→ Bob receives: 9 (thinks it's from Alice)

--- STEP 2: Bob tries to send his public key to Alice ---
Bob sends: B = 4
❌ Tom INTERCEPTS this message!
✓ Tom REPLACES it with his own public key: T = 9
→ Alice receives: 9 (thinks it's from Bob)


In [20]:
print(f"\n{'='*80}")
print("SHARED SECRET COMPUTATION (WITH MITM ATTACK)")
print(f"{'='*80}")

# Alice computes shared secret with what she THINKS is Bob's public key (but it's Tom's)
alice_shared_secret = generate_shared_secret(tom_public_key, alice_private_key, PRIME)
print(f"\n--- Alice's Computation ---")
print(f"Alice receives public key: {tom_public_key} (thinks it's Bob's)")
print(f"Alice computes: {tom_public_key}^{alice_private_key} mod {PRIME} = {alice_shared_secret}")
print(f"Alice's shared secret: {alice_shared_secret}")

# Bob computes shared secret with what he THINKS is Alice's public key (but it's Tom's)
bob_shared_secret = generate_shared_secret(tom_public_key, bob_private_key, PRIME)
print(f"\n--- Bob's Computation ---")
print(f"Bob receives public key: {tom_public_key} (thinks it's Alice's)")
print(f"Bob computes: {tom_public_key}^{bob_private_key} mod {PRIME} = {bob_shared_secret}")
print(f"Bob's shared secret: {bob_shared_secret}")

# Tom computes TWO shared secrets - one with Alice, one with Bob
tom_alice_shared_secret = generate_shared_secret(alice_public_key, tom_private_key, PRIME)
tom_bob_shared_secret = generate_shared_secret(bob_public_key, tom_private_key, PRIME)

print(f"\n--- Tom's Computation (Attacker) ---")
print(f"Tom computes secret with Alice: {alice_public_key}^{tom_private_key} mod {PRIME} = {tom_alice_shared_secret}")
print(f"Tom computes secret with Bob: {bob_public_key}^{tom_private_key} mod {PRIME} = {tom_bob_shared_secret}")


SHARED SECRET COMPUTATION (WITH MITM ATTACK)

--- Alice's Computation ---
Alice receives public key: 9 (thinks it's Bob's)
Alice computes: 9^51 mod 23 = 4
Alice's shared secret: 4

--- Bob's Computation ---
Bob receives public key: 9 (thinks it's Alice's)
Bob computes: 9^48 mod 23 = 6
Bob's shared secret: 6

--- Tom's Computation (Attacker) ---
Tom computes secret with Alice: 17^54 mod 23 = 4
Tom computes secret with Bob: 4^54 mod 23 = 6
